## Chapter1

In [4]:
import os

os.environ["OPENAI_API_KEY"] = "sk-zk24b29e001ee12aed2b92da319b6e15a4f67c6f09ac39e5"
os.environ["TAVILY_API_KEY"] = "your-tavily-api-key"
os.environ["OPENAI_BASE_URL"] = "https://api.zhizengzeng.com/v1"

In [2]:
AGENT_SYSTEM_PROMPT = """
你是一个智能旅行助手，能够帮助用户规划旅行并提供相关信息。

你有以下工具可以使用：
1. get_weather(city): 获取指定城市的天气信息
2. search_attraction(city, weather): 根据城市和天气搜索合适的旅游景点

请按照以下格式回复：
Thought: [你的思考过程]
Action: [要执行的动作，格式为函数调用]

如果不需要使用工具，请直接回复用户。
"""

In [5]:
import requests
import json

def get_weather(city):
    """
    获取指定城市的天气信息
    这里使用模拟数据，实际应用中可以接入真实的天气API
    """
    # 模拟天气数据
    weather_data = {
        "北京": {"temperature": 25, "condition": "晴", "wind": "微风"},
        "上海": {"temperature": 28, "condition": "多云", "wind": "东南风"},
        "广州": {"temperature": 32, "condition": "雷阵雨", "wind": "南风"},
        "深圳": {"temperature": 30, "condition": "阴", "wind": "无风"}
    }
    
    if city in weather_data:
        data = weather_data[city]
        return f"{city}当前天气：{data['condition']}，气温{data['temperature']}°C，{data['wind']}"
    else:
        return f"抱歉，暂时无法获取{city}的天气信息"

In [7]:
# 景点搜索工具

from tavily import TavilyClient
import os

def search_attraction(city, weather):
    """
    根据城市和天气条件搜索合适的旅游景点
    """
    try:
        # 初始化Tavily客户端
        tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
        
        # 构建搜索查询
        query = f"{city} {weather} 适合的旅游景点推荐"
        
        # 执行搜索
        response = tavily.search(query=query, max_results=3)
        
        # 处理搜索结果
        attractions = []
        for result in response.get('results', []):
            attractions.append({
                'title': result.get('title', ''),
                'content': result.get('content', '')[:200] + '...'
            })
        
        return f"根据{city}的{weather}天气，推荐以下景点：" + \
               "\n".join([f"- {attr['title']}: {attr['content']}" for attr in attractions])
    
    except Exception as e:
        return f"搜索景点时出现错误：{str(e)}"

In [10]:
# 可用工具字典
available_tools = {
    "get_weather": get_weather,
    "search_attraction": search_attraction
}

In [8]:
# 实现智能体核心逻辑
from openai import OpenAI
import re
import os

class OpenAICompatibleClient:
    def __init__(self):
        self.client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
            base_url=os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1")
        )
    
    def chat_completion(self, messages, model="gpt-3.5-turbo"):
        """调用LLM进行对话"""
        try:
            response = self.client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0.7,
                max_tokens=1000
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"调用LLM时出现错误：{str(e)}"
    
    def parse_action(self, response):
        """解析LLM响应中的Action"""
        # 使用正则表达式提取Action
        action_pattern = r'Action:\s*([a-zA-Z_]+)\((.*?)\)'
        match = re.search(action_pattern, response)
        
        if match:
            function_name = match.group(1)
            # 简单解析参数（实际应用中需要更robust的解析）
            params_str = match.group(2).strip('"\'')
            params = [p.strip().strip('"\'') for p in params_str.split(',') if p.strip()]
            return function_name, params
        
        return None, None

In [ ]:
def run_agent():
    """运行智能体主循环"""
    client = OpenAICompatibleClient()
    
    # 初始化对话历史
    messages = [{"role": "system", "content": AGENT_SYSTEM_PROMPT}]
    
    print("🤖 智能旅行助手已启动！输入 'quit' 退出。\n")
    
    while True:
        # 获取用户输入
        user_input = input("👤 用户: ").strip()
        if user_input.lower() == 'quit':
            print("👋 再见！")
            break
        
        # 添加用户消息到历史
        messages.append({"role": "user", "content": user_input})
        
        # 开始智能体循环
        max_iterations = 5  # 防止无限循环
        for iteration in range(max_iterations):
            print(f"\n🔄 循环 {iteration + 1}:")
            
            # 1. 思考阶段：调用LLM
            response = client.chat_completion(messages)
            print(f"🧠 智能体响应:\n{response}")
            
            # 2. 解析行动
            function_name, params = client.parse_action(response)
            
            if function_name and function_name in available_tools:
                # 3. 执行行动
                print(f"⚡ 执行工具: {function_name}({', '.join(params)})")
                
                try:
                    tool_result = available_tools[function_name](*params)
                    observation = f"Observation: {tool_result}"
                    print(f"👁️ 观察结果: {tool_result}")
                    
                    # 4. 将观察结果添加到对话历史
                    messages.append({"role": "assistant", "content": response})
                    messages.append({"role": "user", "content": observation})
                    
                except Exception as e:
                    error_msg = f"Observation: 工具执行出错 - {str(e)}"
                    print(f"❌ 错误: {str(e)}")
                    messages.append({"role": "assistant", "content": response})
                    messages.append({"role": "user", "content": error_msg})
            else:
                # 没有工具调用，直接结束循环
                messages.append({"role": "assistant", "content": response})
                print("✅ 任务完成！")
                break
        
        print("\n" + "="*50 + "\n")

In [12]:

# 运行智能体
if __name__ == "__main__":
    run_agent()

🤖 智能旅行助手已启动！输入 'quit' 退出。


🔄 循环 1:
🧠 智能体响应:
我需要先获取北京今天的天气信息，然后根据天气推荐一个合适的旅游景点。

Thought: 首先获取北京今天的天气信息。
Action: get_weather("北京")
⚡ 执行工具: get_weather(北京)
👁️ 观察结果: 北京当前天气：晴，气温25°C，微风

🔄 循环 2:
🧠 智能体响应:
根据北京今天的晴天气温25°C和微风的天气情况，我来为您推荐一个合适的旅游景点。

Thought: 根据北京的晴天气候，推荐一个适合的旅游景点。
Action: search_attraction("北京", "晴")
⚡ 执行工具: search_attraction(北京, 晴)
👁️ 观察结果: 搜索景点时出现错误：Invalid API key: Unauthorized: missing or invalid API key.

🔄 循环 3:
🧠 智能体响应:
抱歉，由于出现了API密钥错误，我无法为您搜索景点。不过，根据北京今天的晴天气候，我可以为您推荐前门、故宫、颐和园等景点，您可以选择其中一个进行游览。希望您在北京旅行愉快！如果您需要更多帮助，请随时告诉我。
✅ 任务完成！


👋 再见！
